In [58]:
MAX_SEQ_LEN = 15


def GlobalParams():
    global MAX_SEQ_LEN

In [59]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import os

In [60]:
# ① 데이터 읽어오기
train_file = os.path.join('./models/intent', "test_sum.csv")
data = pd.read_csv(train_file, delimiter=',')
data

,intent,query
0,0,"음, 그러면 하나 해주세요."
1,1,네. 자동으로 하면 디씨 되나요?
2,2,어떤 게 있나요?
3,2,네. 내추럴 브라운이랑 두 가지 안 되나요?
4,2,어떤 색깔을 주문해야 되나요?
...,...,...
8222,8,요즘은 유행이 되게 단순하게 많이 나가죠?
8223,4,블라우스나 티셔츠 같은거는 얼마부터 있어요?
8224,4,이런 스카프는 얼마예요?
8225,4,이런 옷은 얼마예요?


In [61]:
data['intent'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [62]:
data[data['intent'] == 2]  # 상품색상문의에 대한 문장

,intent,query
2,2,어떤 게 있나요?
3,2,네. 내추럴 브라운이랑 두 가지 안 되나요?
4,2,어떤 색깔을 주문해야 되나요?
5,2,아. 다크가 좀 진한 색이죠?
57,2,어떤 색상 있나요?
...,...,...
8199,2,이 옷은 남색이죠?
8200,2,좀 젊어보이게 입으려면 어떤 색깔이 나아요?
8201,2,이건 블랙도 아니고 챠콜?
8203,2,이 옷 색상은 회색 하나인가요?


In [63]:
queries = data.dropna()['query'].tolist()
intents = data.dropna()['intent'].tolist()

In [64]:
data.dropna()['intent'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [65]:
queries[:20]

['음, 그러면 하나 해주세요.',
 '네. 자동으로 하면 디씨 되나요?',
 '어떤 게 있나요?',
 '네. 내추럴 브라운이랑 두 가지 안 되나요?',
 '어떤 색깔을 주문해야 되나요?',
 '아. 다크가 좀 진한 색이죠?',
 '제가 프라이팬으로 착각했지만 이 상품 박스채 그대로라 반품해주세요.',
 '그거 같은 가격에 살 수 있어요?',
 '네, 주문은 맞는데 취소할려고요.',
 '내가 삼십육이니까.',
 '네. 사만 팔천구백원 아닙니까?',
 '바닥에 하는 거 신청하려구요.',
 '이백삼십오말고 이백사십으로 되나요?',
 '예. 타일 박사 신청하려고요.',
 '네 그런데 칠만 사천 백원으로 뜨는 건 뭐예요?',
 '그러면 취소 좀 시켜주세요.',
 '그러니까 배려를 좀 해주세요.',
 '네. 확인이 안 되면 취소해 주세요.',
 '천원 할인이 적용되는 거죠?',
 '육만구천구백원이요?']

In [66]:
intents[:20]

[0, 1, 2, 2, 2, 2, 3, 4, 5, 6, 4, 0, 6, 0, 4, 5, 5, 5, 1, 4]

In [67]:
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [68]:
sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [69]:
queries[:10]

['음, 그러면 하나 해주세요.',
 '네. 자동으로 하면 디씨 되나요?',
 '어떤 게 있나요?',
 '네. 내추럴 브라운이랑 두 가지 안 되나요?',
 '어떤 색깔을 주문해야 되나요?',
 '아. 다크가 좀 진한 색이죠?',
 '제가 프라이팬으로 착각했지만 이 상품 박스채 그대로라 반품해주세요.',
 '그거 같은 가격에 살 수 있어요?',
 '네, 주문은 맞는데 취소할려고요.',
 '내가 삼십육이니까.']

In [70]:
sequences[:10]

[[320, 708, 2, 685, 25, 244],
 [5, 181, 2, 844, 1019, 13],
 [75, 17, 15],
 [5, 2243, 323, 294, 118, 16, 42, 13],
 [75, 129, 21, 685, 13],
 [7, 1020, 35, 593, 120, 3],
 [63, 1492, 1441, 3, 54, 365, 2244, 536, 178, 10, 685, 25, 244],
 [592, 76, 38, 90, 83, 15],
 [5, 21, 61, 59],
 [212, 119, 157, 3]]

In [71]:
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [72]:
padded_seqs.shape

(8169, 15)

In [73]:
padded_seqs[:10]

array([[ 320,  708,    2,  685,   25,  244,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   5,  181,    2,  844, 1019,   13,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [  75,   17,   15,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   5, 2243,  323,  294,  118,   16,   42,   13,    0,    0,    0,
           0,    0,    0,    0],
       [  75,  129,   21,  685,   13,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   7, 1020,   35,  593,  120,    3,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [  63, 1492, 1441,    3,   54,  365, 2244,  536,  178,   10,  685,
          25,  244,    0,    0],
       [ 592,   76,   38,   90,   83,   15,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   5,   21,   61,   59,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [ 212,  119,  157,    3,    0,

In [74]:
# ③ 학습용, 검증용, 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents)) # 패딩처리된 시퀀스와 의도(intent) 리스트 전체를 데이터셋 객체로
ds = ds.shuffle(len(queries)) # 랜덤 섞기

# 학습셋:검증셋:테스트셋 = 7:2:1
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수

In [75]:
# ④ CNN 모델 정의
# keras 함수형 모델 방식으로 구현
input_layer = Input(shape=(MAX_SEQ_LEN,))  # 입력크기
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(10, name='logits')(dropout_hidden)  # 최종적으로 10개의 의도 클래스를 분류. 결과로 나온 값(logits) 을을 점수(score) 라 부른다
predictions = Dense(10, activation=tf.nn.softmax)(logits)

In [76]:
# ⑤ 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [77]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 15, 128)      374656      ['input_2[0][0]']                
                                                                                                  
 dropout_2 (Dropout)            (None, 15, 128)      0           ['embedding_1[0][0]']            
                                                                                                  
 conv1d_3 (Conv1D)              (None, 13, 128)      49280       ['dropout_2[0][0]']              
                                                                                            

In [78]:
# 모델 학습
# ★ 시간 걸림 ★

model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
286/286 [==============================] - 3s 8ms/step - loss: 1.1013 - accuracy: 0.6441 - val_loss: 0.4492 - val_accuracy: 0.8910
Epoch 2/5
286/286 [==============================] - 2s 7ms/step - loss: 0.4367 - accuracy: 0.8900 - val_loss: 0.2451 - val_accuracy: 0.9369
Epoch 3/5
286/286 [==============================] - 2s 7ms/step - loss: 0.2948 - accuracy: 0.9246 - val_loss: 0.1609 - val_accuracy: 0.9608
Epoch 4/5
286/286 [==============================] - 2s 7ms/step - loss: 0.2387 - accuracy: 0.9405 - val_loss: 0.1225 - val_accuracy: 0.9706
Epoch 5/5
286/286 [==============================] - 2s 7ms/step - loss: 0.1950 - accuracy: 0.9488 - val_loss: 0.0973 - val_accuracy: 0.9731


In [79]:
# ⑦ 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

41/41 [==============================] - 0s 2ms/step - loss: 0.1386 - accuracy: 0.9596
Accuracy: 95.955884
loss: 0.138594


In [80]:
# ⑧ 모델 저장
model.save(os.path.join('./models/intent','intent_model.h5'))

In [81]:
from utils.Preprocess import Preprocess    # 의도분류 모델 테스트
from models.intent.IntentModel import IntentModel
import os

p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'user_dic_1.tsv'))

In [82]:
intent = IntentModel(model_name=os.path.join('./models/intent', 'intent_model.h5'), preprocess=p)

In [83]:
query = '뭐가 잘나가요?'
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

뭐가 잘나가요?
의도 예측 클래스 :  8
의도 예측 레이블 :  제품
